# CoppeliaSim: PART 2
* Run the scene workshop1.ttt in CoppeliaSim

In [85]:
import sim
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Check if python is connecting to Coppelia

In [86]:
sim.simxFinish(-1)  # Close opened connections
clientID = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim

if clientID != -1:
    print('Connected')

    # Now try to retrieve data in a blocking fashion (i.e. a service call):
    res, objs = sim.simxGetObjects(clientID, sim.sim_handle_all, sim.simx_opmode_blocking)

    print('Simulation time in milliseconds: ', sim.simxGetLastCmdTime(clientID))
    
    # Get Object position
    name = 'Omnirob'
    err_code, cuboid = sim.simxGetObjectHandle(clientID, name, sim.simx_opmode_blocking)
    res, position = sim.simxGetObjectPosition(clientID, cuboid, -1, sim.simx_opmode_blocking)        
    print('Omnirob is at [x,y,z]=', position)
    
    # Now close the connection to CoppeliaSim:
    sim.simxGetPingTime(clientID)
    sim.simxFinish(clientID)
    print('Disconnected')
else:
    print('Failed connecting to remote API server')


Connected
Simulation time in milliseconds:  1200
Omnirob is at [x,y,z]= [7.62499475479126, 8.549812316894531, 0.35449883341789246]
Disconnected


# Robot

In [87]:
class robot():
    
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart('127.0.0.1', 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)

# 1.3 Number 3

In [88]:
# Derived
a0_pairs = [[7.6250,8.5500], [0.4000, 8.0500], [4.6000,6.6000], [1.9250, 2.2250], 
            [7.2500,2.2250], [4.8000,-2.5000], [.7755,-6.5250], [6.1250,-6.1500], 
            [8.8500,-2.5000], [4.8000,-2.5000], [-1.2000,-3.0750],
            [-3.7500, 0.0000], [-3.7500,5.0000], [-7.8500,5.2250], 
            [-5.0750,-4.5750], [-3.4500,-7.0750]]

a1_pairs = [[0, 0], [0,.5], [.25,-.75], [-.25,.25], [.1,0], [-1,0], 
            [.5,0], [1.5,0], [0,.75], [-1,0], [-.1,1], [-.05,1], [0,1],
            [.5,-1], [1,-.25], [0,-.5], [0,0]]

a2_pairs = [[-0.2167, -0.0650], [0.1010,-0.0685], [-0.1052,-0.0062], 
            [0.1998,-0.0500], [0.0065,-0.1418], [0.0293,-0.1208], 
            [-0.0895,0.0112], [-0.2183,0.0345], [-0.0215,-0.1500], 
            [0.0300,-0.1173], [-0.0515,-0.2077],[0.0100,-0.1500],
            [-0.1730,-0.0933], [-0.1168,-0.0690], [-0.1512,0.0250], 
            [-0.0960,0.0573]]

a3_pairs = [[0.0144,0.0060],[-0.0059,0.0004],[0.0053,0.0037], [-0.0122,0.0025],
            [-0.0041,0.0095],[0.0030,0.0080],[0.0093,-0.0008],[0.0096,0.0002],
            [-0.0019,0.0075],[0.0010,0.0112],[0.0036,0.0138], [-0.0005,0.0100],
            [0.0132,-0.0004],[0.0095,0.0071],[0.0067,-0.0025], 
            [0.0064,-0.0021]]

waypoints = ['Sphere1', 'Checkpoint1', 'Sphere', 'Checkpoint2', 'Sphere2', 
             'Checkpoint3', 'Sphere3', 'Checkpoint4', 'Checkpoint5', 
             'Checkpoint6', 'Checkpoint7', 'Sphere0', 'Checkpoint8', 
             'Checkpoint9', 'Checkpoint10', 'Sphere4']

In [ ]:
# Joint name in coppelia
motor_names = ['Omnirob_FLwheel_motor', 'Omnirob_FRwheel_motor', 'Omnirob_RRwheel_motor', 'Omnirob_RLwheel_motor']
r = robot('Omnirob', motor_names)  # Create an instance of our robot

# Initial locations
robot_position = r.get_position()
print('Robot position: (%.2f, %.2f) '%(robot_position[0], robot_position[1]))

# Virtual disk
disk = robot('Disc', client_id=r.client_id)

# Time duration
tf = 10
# Time interval
time_steps = linspace(0, tf, 500)

count = 0
for wp in waypoints:  
    print("Count: ",count)
    a0 = a0_pairs[count]
    a1 = a1_pairs[count]
    a2 = a2_pairs[count]
    a3 = a3_pairs[count]
    print('a0: %s'%(a0))
    print('a1: %s'%(a1))
    print('a2: %s'%(a2))
    print('a3: %s'%(a3))

    for t in time_steps:
        # Compute trajectory
        x_point_traj = a0[0] + a1[0]*t + a2[0]*(t**2) + a3[0]*(t**3) #Gamma_x
        x_vel_traj = a1[0] + 2*a2[0]*t + 3*a3[0]*(t**2) #Derivative of Gamma_x
        y_point_traj = a0[1] + a1[1]*t + a2[1]*(t**2) + a3[1]*(t**3) #Gamma_y
        y_vel_traj = a1[1] + 2*a2[1]*t + 3*a3[1]*(t**2) #Derivative of Gamma_y
        print('(%.4f,%.4f)' %(x_point_traj, y_point_traj))

        disk.set_position([x_point_traj, y_point_traj])

        # Sensing
        robot_position = r.get_position()

        # Trajectory tracker
        u = 10 * (np.array([x_point_traj, y_point_traj, 0]) - robot_position) + np.array([x_vel_traj, y_vel_traj, 0])

        vx, vy, vz = u
        r.send_motor_velocities([-vy - vx, vy - vx, vy + vx, -vy + vx])

        time.sleep(tf/500)
    
    count+=1

    r.send_motor_velocities([0,0,0,0])
r.close_connection()

Robot connected
Robot position: (7.62, 8.55) 
Robot connected
Count:  0
a0: [7.625, 8.55]
a1: [0, 0]
a2: [-0.2167, -0.065]
a3: [0.0144, 0.006]
(7.6250,8.5500)
(7.6249,8.5500)
(7.6247,8.5499)
(7.6242,8.5498)
(7.6236,8.5496)
(7.6228,8.5494)
(7.6219,8.5491)
(7.6208,8.5487)
(7.6195,8.5484)
(7.6180,8.5479)
(7.6164,8.5474)
(7.6146,8.5469)
(7.6127,8.5463)
(7.6105,8.5457)
(7.6083,8.5450)
(7.6058,8.5443)
(7.6032,8.5435)
(7.6004,8.5427)
(7.5975,8.5418)
(7.5944,8.5409)
(7.5911,8.5399)
(7.5877,8.5389)
(7.5841,8.5379)
(7.5804,8.5368)
(7.5765,8.5356)
(7.5724,8.5344)
(7.5682,8.5332)
(7.5638,8.5319)
(7.5593,8.5306)
(7.5546,8.5292)
(7.5498,8.5278)
(7.5448,8.5264)
(7.5397,8.5249)
(7.5344,8.5233)
(7.5290,8.5217)
(7.5234,8.5201)
(7.5176,8.5184)
(7.5117,8.5167)
(7.5057,8.5150)
(7.4995,8.5132)
(7.4932,8.5113)
(7.4867,8.5094)
(7.4801,8.5075)
(7.4733,8.5056)
(7.4664,8.5036)
(7.4593,8.5015)
(7.4521,8.4995)
(7.4448,8.4973)
(7.4373,8.4952)
(7.4297,8.4930)
(7.4219,8.4908)
(7.4140,8.4885)
(7.4060,8.4862)
(7.3978,8